<a href="https://colab.research.google.com/github/sviii09/Comparative-Study-AlexNet-VGG-ResNet/blob/main/DNA_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Comparative Study: AlexNet, VGG, ResNet
# Dataset: CIFAR-10

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

classes = trainset.classes
num_classes = len(classes)

def train_and_test(model, model_name, epochs=2):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"[{model_name}] Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader):.4f}")

    train_time = time.time() - start_time

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    params = sum(p.numel() for p in model.parameters())

    print(f"[{model_name}] Accuracy: {accuracy:.2f}% | Time: {train_time:.2f}s | Parameters: {params:,}")
    return accuracy, train_time, params

models_to_test = {
    "AlexNet": models.alexnet(pretrained=True),
    "VGG16": models.vgg16(pretrained=True),
    "ResNet18": models.resnet18(pretrained=True)
}

results = {}
for name, model in models_to_test.items():
    if name == "ResNet18":
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    else:
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, num_classes)

    acc, ttime, params = train_and_test(model, name)
    results[name] = (acc, ttime, params)

print("\nFinal Comparison:")
print("Model     | Accuracy | Time (s) | Parameters")
for name, (acc, ttime, params) in results.items():
    print(f"{name:8} | {acc:.2f}%   | {ttime:.2f}    | {params:,}")


Using device: cuda


100%|██████████| 170M/170M [00:04<00:00, 39.6MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 182MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'we

[AlexNet] Epoch 1/2, Loss: 1.3950
[AlexNet] Epoch 2/2, Loss: 0.9686
[AlexNet] Accuracy: 71.18% | Time: 259.19s | Parameters: 57,044,810
[VGG16] Epoch 1/2, Loss: 1.8758
[VGG16] Epoch 2/2, Loss: 1.4093
[VGG16] Accuracy: 55.64% | Time: 1528.21s | Parameters: 134,301,514
[ResNet18] Epoch 1/2, Loss: 0.5588
[ResNet18] Epoch 2/2, Loss: 0.3175
[ResNet18] Accuracy: 86.34% | Time: 444.33s | Parameters: 11,181,642

Final Comparison:
Model     | Accuracy | Time (s) | Parameters
AlexNet  | 71.18%   | 259.19    | 57,044,810
VGG16    | 55.64%   | 1528.21    | 134,301,514
ResNet18 | 86.34%   | 444.33    | 11,181,642
